In [2]:
%matplotlib inline
from __future__ import division

import pandas as pd
import matplotlib.pyplot as plt
import utils

In [3]:
visitors, events, devices, url_categories = utils.load_data(event_categories=True)

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5,6,7,8,11,12,13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
sample_npis = list(visitors[(visitors.professional_designation=='Physician')\
                       &(~visitors.npi_number.isnull())\
                       &(visitors.exclude==0)].npi_number.sample(10000))

In [31]:
def categorize_frequency(n):
    if n < 1:
        return 'Less than 1 visit a month' 
    elif n <= 10:
        return '1-10 times per month'
    else:
        return 'More than 10x per month'
    
def npi_level_summary_statistics(output_file, npis=None):
    # if given a list of users, pull in those. otherwise, use all available data
    if npis:
        v = visitors[visitors.npi_number.isin(npis)]
    else:
        v = visitors[visitors.exclude==0]
    ev = pd.merge(v, events, on='dg_id')
    if len(ev) == 0:
        print 'target list has no web activity'
        return
    # check which users have web activity
    event_dgids = set(ev.dg_id)
    v['has_event'] = v.dg_id.apply(lambda x: 1 if x in event_dgids else 0)
    
    # check frequency of web activity (timeframe denominator starts with the earliest date the user has activity)
    max_dt = max(events.timestamp).date()
    ev['dt'] = ev.timestamp.apply(lambda x: x.date())
    freq = ev.drop_duplicates(subset=['dg_id', 'dt']).groupby(['dg_id']).dt.agg(['min', 'max', 'count']).reset_index()
    freq['sessions_per_month'] = freq.apply(lambda x: x['count']/(max(1, (max_dt-x['min']).days)/30), axis=1)
    freq['frequency_category'] = freq.sessions_per_month.apply(lambda x: categorize_frequency(x))
    v = pd.merge(v, freq[['dg_id', 'sessions_per_month', 'frequency_category']], how='left')
    
    # grab avg number of urls per session
    urls_per_session = ev.groupby(['dg_id', 'session_id']).url.nunique().reset_index()\
        .groupby('dg_id').url.mean().reset_index()
    urls_per_session = urls_per_session.rename(columns={'url':'urls_per_session'})
    v = pd.merge(v, urls_per_session, on='dg_id', how='left')
    
    tableau_columns = [
        'dg_id', 
        'identity_type', 
        'professional_designation', 
        'npi_number', 
        'primary_specialty', 
        'primary_specialty_group', 
        'birth_year', 
        'grad_year', 
        'gender', 
        'has_event', 
        'sessions_per_month', 
        'frequency_category', 
        'urls_per_session' 
    ]
    
    v[tableau_columns].to_csv(output_file)

In [24]:
npis = list(v[~v.npi_number.isnull()].npi_number.sample(5))

In [32]:
npi_level_summary_statistics('../data/testest.csv', npis=npis)

target list has no web activity


In [41]:
event_npis = list(visitors[(visitors.dg_id.isin(events.dg_id.sample(5)))&(~visitors.npi_number.isnull())].npi_number)

In [43]:
npi_level_summary_statistics('../data/testest.csv', npis=event_npis)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
